# 🐕 DogTrain AI - Launchpad

Run this notebook to launch your Dog Training App directly from Google Drive.

### Instructions:
1.  Run the **Setup** cell below (click the Play button).
2.  Follow the prompts to authorize Google Drive access.
3.  Enter your **Gemini API Key** when asked.
4.  Click the generated **Public URL** to open the app!

In [1]:
# @title 1. Setup & Launch
import os
import shutil
import subprocess
import time
from google.colab import drive
from google.colab import userdata

# 1. Mount Google Drive
print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Navigate to Project Directory
PROJECT_PATH = '/content/drive/My Drive/dog-training-app'
if not os.path.exists(PROJECT_PATH):
    print(f"❌ Error: Project folder not found at {PROJECT_PATH}")
    print("Please make sure the 'dog-training-app' folder is in the root of your My Drive.")
else:
    os.chdir(PROJECT_PATH)
    print(f"✅ Working directory set to: {PROJECT_PATH}")

    # 3. Install Dependencies
    print("📦 Installing dependencies... (this may take a minute)")
    subprocess.run(['pip', 'install', '-r', 'backend/requirements.txt'], stdout=subprocess.DEVNULL)
    subprocess.run(['pip', 'install', 'pyngrok', 'uvicorn'], stdout=subprocess.DEVNULL)
    print("✅ Dependencies installed.")

    # 4. Prepare Frontend
    print("🎨 Preparing frontend...")
    frontend_dist = os.path.join(PROJECT_PATH, 'frontend', 'dist')
    backend_static = os.path.join(PROJECT_PATH, 'backend', 'static')
    
    if os.path.exists(frontend_dist):
        # Clear existing static files (except index.html if it was the lite version, but we overwrite anyway)
        if os.path.exists(backend_static):
            shutil.rmtree(backend_static)
        shutil.copytree(frontend_dist, backend_static)
        print("✅ Frontend build copied to backend.")
    else:
        print("⚠️ Frontend build not found. Using existing static files (if any).")

    # 5. Configure API Key
    print("🔑 Configuration")
    try:
        gemini_key = userdata.get('GEMINI_API_KEY')
    except:
        gemini_key = input("Enter your Gemini API Key: ")
    
    os.environ['GEMINI_API_KEY'] = gemini_key
    os.environ['PYTHONPATH'] = os.path.join(PROJECT_PATH, 'backend')

    # 6. Start Backend
    print("🚀 Starting Server...")
    # Kill any previous uvicorn
    subprocess.run(['pkill', '-f', 'uvicorn'])
    
    # Start uvicorn in background
    log_file = open('server.log', 'w')
    process = subprocess.Popen(
        ['python3', '-m', 'uvicorn', 'app.main:app', '--host', '0.0.0.0', '--port', '8000'],
        cwd=os.path.join(PROJECT_PATH, 'backend'),
        stdout=log_file,
        stderr=log_file,
        env=os.environ
    )
    
    time.sleep(5) # Wait for startup

    # 7. Expose via Ngrok (or Localtunnel)
    # Using localtunnel for ease of use (no token needed)
    print("🌐 Creating Public URL...")
    subprocess.run(['npm', 'install', '-g', 'localtunnel'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    lt_process = subprocess.Popen(['lt', '--port', '8000'], stdout=subprocess.PIPE)
    time.sleep(3)
    
    # Try to grab the URL
    # Note: localtunnel often requires a password (IP address) now. 
    # Let's use pyngrok if possible, but it needs a token. 
    # Fallback: Use google.colab.output.serve_kernel_port_as_iframe for local-only, 
    # OR just print instructions for localtunnel.
    
    print("\n" + "="*40)
    print("🎉 APP IS RUNNING!")
    print("="*40)
    
    from google.colab.output import eval_js
    print(f"Click here to open: {eval_js('google.colab.kernel.proxyPort(8000)')}")
    print("Note: This link only works for YOU (the person running the notebook).")
    print("To share with others, you would need to set up Ngrok with an auth token.")
    
    # Keep cell running
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("🛑 Stopping server...")
        process.terminate()
        lt_process.terminate()


ModuleNotFoundError: No module named 'google.colab'